In [7]:
import pandas as pd
import numpy as np
import heapq
import pdb
import math, sys
from scipy.stats import chi2_contingency, chi2
from collections import defaultdict
from itertools import combinations, chain

In [39]:
## Set up test case for particular node
df = pd.read_csv("tests/data-strikes2/data.txt", delim_whitespace=True)
GOTO_NODE = True
if GOTO_NODE:
    _df = df.loc[df['num1'] <= 1.5]
    _df = _df.loc[_df['num2'] <= 0.4375]
    _df = _df.loc[_df['num1'] <= 0.5]
######################################################################################

col = 'cat1'
self_tgt = 'tgt'
node_y_mean = _df[self_tgt].mean()
_df = _df[[col, self_tgt]]
x_uniq = _df[col].drop_duplicates().sort_values().values

node_binom = _df[self_tgt] <= node_y_mean                      # class 1 has negative residual
_df.loc[:,self_tgt] = node_binom

def pi(j):
    """ prior probability of the class in the node """
    if j == 1: return _df[self_tgt].mean()
    # if j == 1: return (df[self_tgt] < df[self_tgt].mean()).mean()
    if j == 0: return 1.0 - pi(1)
    raise "fail"
    
def N_j_l(j,s):
    """ num cases of class j in subset s"""
    assert j == 0 or j == 1
    return _df[(_df[self_tgt] == j) & (_df[col]== s)].shape[0]
    
def p(j, subset_l) -> float:
    """ weighted probability of class j in the subset?? """
    assert j == 0 or j == 1
    return pi(j)*N_j_l(j, subset_l) / ( pi(1)*N_j_l(1, subset_l) + pi(0)*N_j_l(0, subset_l) )


sorted_ps = sorted([(s, p(1, s)) for s in x_uniq],key=lambda x:x[1])
sorted_ps = [s for s,_ in sorted_ps]
st = 0
en = 0
subsets = [sorted_ps[st:en] for en in range(1,len(x_uniq))]


In [40]:
subsets

[['A'],
 ['A', 'J'],
 ['A', 'J', 'H'],
 ['A', 'J', 'H', 'G'],
 ['A', 'J', 'H', 'G', 'D'],
 ['A', 'J', 'H', 'G', 'D', 'E'],
 ['A', 'J', 'H', 'G', 'D', 'E', 'F'],
 ['A', 'J', 'H', 'G', 'D', 'E', 'F', 'I'],
 ['A', 'J', 'H', 'G', 'D', 'E', 'F', 'I', 'B']]